## 3.3 데이터 전처리

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

# 작업 경로 설정
import os
os.chdir(r"/content/gdrive/My Drive/Colab Notebooks/week4/ch03")

Mounted at /content/gdrive


In [2]:
import pandas as pd

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
bts= pd.read_csv('bus_bts.csv')
jeju_life = pd.read_csv('jeju_financial_life_data.csv')
weather = pd.read_csv('weather.csv', encoding='cp949')
rain = pd.read_csv('rain.csv', encoding='utf-8')

In [3]:
# 시각화에 필요한 라이브러리 임포트
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# 내부에 결과를 출력하도록 설정
%matplotlib inline
# 시각화 한글 폰트 설정
# 윈도우 사용자
plt.rc('font', family='Malgun Gothic')
# Mac 사용자
#plt.rc('font', family='AppleGothic')
# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False) 
# 분석에 문제가 없는 경고 메세지는 숨긴다.
import warnings
warnings.filterwarnings('ignore') 

In [4]:
# 학습 데이터와 테스트 데이터를 구분하기 위한 변수 생성
train['cue'] = 0
test['cue'] = 1

# 학습 데이터와 테스트 데이터 통합
df = pd.concat([train, test], axis=0)

### 3.3.1 내부 데이터를 통한 변수 생성

#### 3.3.1.1 탐색적 데이터 분석을 통한 변수

* 요일을 나타내는 변수(weekday)

In [ ]:
# datetime 변수형으로 변환
df['date'] = pd.to_datetime(df['date'])
# 요일 추출 (0-월요일~6-일요일)
df['weekday'] = df['date'].dt.weekday

In [ ]:
df[['weekday']].head()

,weekday
0,6
1,6
2,6
3,6
4,6


* 요일별 평균 탑승객 수

In [ ]:
# 요일별 평균 탑승 승객수를 구하는 함수
def week_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 'weekdaymean'변수 생성
    df['weekdaymean'] = 1

    # 각 요일에 해당하는 인덱스 추출
    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    # 인덱스를 활용하여 'weekdaymean'의 값을 각 요일에 맞는 평균 탑승 승객수로 변경 
    df.iloc[index0,-1] = train_data.query('weekday==0')['18~20_ride'].mean()
    df.iloc[index1,-1] = train_data.query('weekday==1')['18~20_ride'].mean()
    df.iloc[index2,-1] = train_data.query('weekday==2')['18~20_ride'].mean()
    df.iloc[index3,-1] = train_data.query('weekday==3')['18~20_ride'].mean()
    df.iloc[index4,-1] = train_data.query('weekday==4')['18~20_ride'].mean()
    df.iloc[index5,-1] = train_data.query('weekday==5')['18~20_ride'].mean()
    df.iloc[index6,-1] = train_data.query('weekday==6')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = week_mean()

In [ ]:
df[['weekdaymean']].head()

,weekdaymean
0,1.034282
1,1.034282
2,1.034282
3,1.034282
4,1.034282


* 버스 종류별 평균 탑승객 수

In [ ]:
# 버스 종류별 평균 탑승 승객수를 구하는 함수
def inout_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 in_out_mean 변수 생성
    df['in_out_mean'] = 1

    # 버스 종류별 인덱스 추출
    in_index = df.query('in_out == "시내"').index # 시내버스
    out_index = df.query('in_out == "시외"').index # 시외버스

    # 인덱스를 활용하여 in_out_mean의 값을 각 버스 종류에 맞는 
    # 평균 탑승 승객수로 변경
    df.iloc[in_index,-1] = train_data.query('in_out == "시내"')['18~20_ride'].mean()
    df.iloc[out_index,-1] = train_data.query('in_out == "시외"')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = inout_mean()

In [ ]:
df[['in_out_mean']].head()

,in_out_mean
0,2.044345
1,2.044345
2,2.044345
3,1.228499
4,1.228499


* 일별 오전 시간대의 총 탑승객 수

In [ ]:
# 날짜별 오전시간에 탑승한 총 승객수
f=df.groupby('date')['6~7_ride','7~8_ride','8~9_ride','9~10_ride','10~11_ride'
                    ].sum().reset_index()

# 변수명 바꾸기
f.columns=['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
           '10~11_ride_sum']

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, f, how='left', on='date')

In [ ]:
df[['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
    '10~11_ride_sum']].head()

,date,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum
0,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
1,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
2,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
3,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
4,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0


#### 3.3.1.2 도메인 조사를 통한 변수

* 배차 간격

In [ ]:
# 탑승 날짜를 datetime형으로 변환
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

# 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호 기준으로 정렬
f = bts.sort_values(by=['geton_date','geton_station_code','bus_route_id',
                        'geton_time2'],ascending=True).reset_index()
                        

In [ ]:
print(f)

In [ ]:
# 반복문 실행 시간이 많이 소요되어 상태 진행률을 확인할 수 있는 tqdm 라이브러리 사용
from tqdm import notebook

# 시차를 넣기 위한 빈 리스트 생성
interval_lst = []
for i in notebook.tqdm(range(0,len(f)-1)):
    # 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호가 같다면
    if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) & 
        (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) & 
        (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):
        # 탑승 날짜의 차를 interval_lst에 넣기
        interval_lst.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)
    else:
       # 같지 않다면 0을 interval_lst에 넣기
        interval_lst.append(0)

interval_lst.insert(0, '0')

In [ ]:
f['interval'] = interval_lst # interval_lst를 'interval'변수로 생성 
# 'interval' 변수의 값을 문자형으로 변환 후 '시:분:초'의 형태로 변환
f['interval'] = f['interval'].astype(str).str[7:].replace('','00:00:00')
f['bus_route_id'] = f['bus_route_id'].astype(object)

In [ ]:
# 분으로 변환하는 함수
def get_min(time_str):
    h, m, s = time_str.split(':')
    return (int(h) * 3600 + int(m) * 60 + int(s)) / 60
# 'interval' 함수의 모든 값에 대해 함수 적용하여 나온 값을 time리스트에 넣기
time = []
for i in f['interval'] :
    time.append(get_min(i))

f['time'] = time # time리스트를 'time' 변수로 생성
f['time'] = f['time'].astype(int) # 'time' 변수를 int 타입으로 변경
f = f[f['time'] > 3] # 간격이 3분보다 작은 것 제외 
f = f[f['time'] < 180] # 간격이 3시간보다 큰 것 제외

In [ ]:
# 버스 노선별 버스 간격의 평균 'bus_interval’변수 생성
data = f.groupby('bus_route_id')['time'].agg([('bus_interval', 'mean')]
                                            ).reset_index()
data['bus_interval'] = data['bus_interval'].astype(int)

# 변수 생성에 많은 시간이 소요되므로 csv 파일로 저장
data.to_csv('bus_interval.csv', index = False)

In [ ]:
# csv 파일이 성공적으로 저장됐다면 다음 과정부터 진행
import numpy as np
df['bus_route_id'] = df['bus_route_id'].astype(np.int64)
bus_interval = pd.read_csv("bus_interval.csv")

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, bus_interval, how = 'left', on = 'bus_route_id')
df['bus_interval'] = df['bus_interval'].fillna(9999) # 병합 중 생기는 NA값 처리

In [ ]:
df[['bus_interval']].head()

,bus_interval
0,34.0
1,34.0
2,34.0
3,34.0
4,34.0


* 수요가 많을 것으로 예상되는 정류장

In [ ]:
# school 변수 생성
# 정류장명에 '고등학교'가 포함되어 있는 행 뽑기 
highschool_station = df[df['station_name'].str.contains('고등학교')]
# '고등학교'가 포함되어 있는 정류장명을 중복 없이 리스트에 저장
highschool = list(highschool_station['station_name'].unique())
# 정류장명에 '대학교'가 포함되어 있는 행 뽑기
university_station = df[df['station_name'].str.contains('대학교')]
# '대학교'가 포함되어 있는 정류장명을 중복 없이 리스트에 저장
university = list(university_station['station_name'].unique())

# '고등학교'와 '대학교' 근처 정류장 구분 함수
def schoolstation(x):
    # highschool 리스트에 정류장명이 포함되어 있다면 1을 반환
    if x in highschool:
        return 1
    # university 리스트에 정류장명이 포함되어 있다면 1을 반환
    elif x in university:
        return 1
    # 둘 다 포함되지 않는 정류장명이라면 0을 반환
    else:
        return 0

# 모든 정류장명에 대해 적용
df['school'] = df['station_name'].apply(schoolstation) 

In [ ]:
df[['school']].head()

In [ ]:
# school 변수와 동일한 방법으로 transfer 변수 생성
transfer_station = df[df['station_name'].str.contains('환승')]
transfers = list(transfer_station['station_name'].unique())

airport_station = df[df['station_name'].str.contains('공항')]
airport = list(airport_station['station_name'].unique())

terminal_station = df[df['station_name'].str.contains('터미널')]
terminal = list(terminal_station['station_name'].unique())

# 항상 수요가 많은 정류장 구분 함수
def transfer_station(x):
    if x in transfers:
        return 1
    elif x in airport:
        return 1
    elif x in terminal:
        return 1
    else:
        return 0

df['transfer'] = df['station_name'].apply(transfer_station)

In [ ]:
df[['transfer']].head()

* 연휴

In [ ]:
# 날짜에서 연휴 구분 함수
def holiday(x):
    # 공휴일이라면 1을 반환
    if x in ['2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09']:
        return 1
    else:
        # 공휴일이 아니라면 0을 반환
        return 0

    # 모든 날짜에 적용
df['holiday'] = df['date'].apply(holiday)
df[['holiday']].head()

,holiday
0,0
1,0
2,0
3,0
4,0


#### 3.3.1.3. 시간대를 활용한 변수

* 승하차 시간대 통합 변수(t~t+2)

In [ ]:
# 오전 시간대에 2시간 동안 승차한 인원수의 합계
df['68a'] = df['6~7_ride'] + df['7~8_ride'] 
df['810a'] = df['8~9_ride'] + df['9~10_ride']
df['1012a'] = df['10~11_ride'] + df['11~12_ride']

# 오전 시간대에 2시간 동안 하차한 인원수의 합계
df['68b'] = df['6~7_takeoff'] + df['7~8_takeoff'] 
df['810b'] = df['8~9_takeoff'] + df['9~10_takeoff']
df['1012b'] = df['10~11_takeoff'] + df['11~12_takeoff']

In [ ]:
df[['68a', '810a', '1012a', '68b', '810b', '1012b']].head()

,68a,810a,1012a,68b,810b,1012b
0,1.0,7.0,8.0,0.0,0.0,0.0
1,5.0,6.0,11.0,0.0,0.0,0.0
2,2.0,2.0,0.0,0.0,0.0,0.0
3,17.0,32.0,30.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0


* 오전 시간의 승객 수

In [ ]:
# 오전 시간(6시~12시)에 승차한 승객 수의 합계
df['ride_sum'] = df[['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', 
                     '10~11_ride', '11~12_ride']].sum(axis=1)

# 오전 시간(6시~12시)에 하차한 승객 수의 합계
df['takeoff_sum'] = df[['6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff',
                        '10~11_takeoff', '11~12_takeoff']].sum(axis=1)

In [ ]:
df[['ride_sum', 'takeoff_sum']].head()

,ride_sum,takeoff_sum
0,16.0,0.0
1,22.0,0.0
2,4.0,0.0
3,79.0,0.0
4,0.0,1.0


In [ ]:
# 'bus_route_id'와 'station_code'를 결합해 route_station 생성
df['route_station'] = df['bus_route_id'].astype(str) + ',' \
                    + df['station_code'].astype(str)

# 버스노선별 오전시간의 총 승객수와 평균 승객수 구하기
data = df.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()

# 변수명을 route_station, 1012a_mean, 1012a_sum으로 바꾸기
data.rename(columns = {'mean':'1012a_mean', 'sum':'1012a_sum'}, inplace=True)

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, data, how='left', on='route_station')

In [ ]:
df[['1012a_mean','1012a_sum']].head()

,1012a_mean,1012a_sum
0,5.304348,244.0
1,11.608696,534.0
2,2.304348,106.0
3,36.673913,1687.0
4,0.564103,22.0


#### 3.3.1.4. bus_bts를 활용한 변수

* 카테고리별 승객 수의 합과 비율

In [ ]:
# 카테고리별 승객 수의 합
def category_people():
    # df와 병합을 위해 bus_route_id의 타입을 str로 변환
    bts['bus_route_id'] = bts['bus_route_id'].astype(str)

    # bus_route_id와 user_category별 승객 수의 합
    f = bts.groupby(
        ['bus_route_id','user_category']
    )['user_count'].agg([('승객수', 'sum')]).reset_index()

    # bus_route_id별 user_category 변수의 승객수의 피벗 테이블 형태로 변환
    g = pd.pivot_table(f, values='승객수', index='bus_route_id', 
                       columns='user_category', fill_value=0).reset_index()
    g.columns = ['bus_route_id', 'adult','kids','teen','elder','jang','jang2',
                 'ugong','ugong2']
    g = g[['bus_route_id', 'adult','kids','teen','elder']]

    # 기존의 데이터프레임에 새로운 변수를 병합
    merge_data = pd.merge(df, g, how='left', on='bus_route_id')
                 
    return merge_data

# bus_route_id의 타입을 str로 통일시켜준 뒤, 함수를 실행하여 변수 생성
df['bus_route_id'] = df['bus_route_id'].astype(str)
df = category_people()

In [ ]:
df[['adult','kids','teen','elder']].head()

,adult,kids,teen,elder
0,8158.0,72.0,38.0,0.0
1,8158.0,72.0,38.0,0.0
2,8158.0,72.0,38.0,0.0
3,8158.0,72.0,38.0,0.0
4,8158.0,72.0,38.0,0.0


In [ ]:
# 카테고리별 승객 비율
def category_people_ratio() :

    # bus_route_id별 user_category의 전체 합
    f_all = bts.groupby('bus_route_id'
                       )['user_count'].agg([('전체', 'sum')]).reset_index()

   # bus_route_id와 user_category별 승객 수의 합
    f_sum = bts.groupby(['bus_route_id','user_category']
                       )['user_count'].agg([('승객수', 'sum')]).reset_index()
    
    # 승객 수의 합을 전체 합으로 나눠줘 비율을 구함
    data = pd.merge(f_all, f_sum, on='bus_route_id')
    data['비율'] = data['승객수'] / data['전체']

   # 피벗 테이블을 통해 bus_route_id별 user_category변수의 승객 비율 형태로 변환
    data = pd.pivot_table(data, values='비율', index='bus_route_id', 
                          columns='user_category',fill_value=0).reset_index()
    data.columns = ['bus_route_id', 'adult_prop','kids_prop','teen_prop',
                    'elder_prop','jang_prop','jang2_prop','ugong_prop','ugong2_prop']
    f = data[['bus_route_id', 'adult_prop','kids_prop','teen_prop','elder_prop']]

    # 기존의 데이터프레임에 새로운 변수를 병합
    merge_data = pd.merge(df, f, how='left', on='bus_route_id')

    return merge_data

# 함수를 실행하여 변수 생성
df = category_people_ratio()

In [ ]:
df[['adult_prop','kids_prop','teen_prop','elder_prop']].head()

,adult_prop,kids_prop,teen_prop,elder_prop
0,0.986696,0.008708,0.004596,0.0
1,0.986696,0.008708,0.004596,0.0
2,0.986696,0.008708,0.004596,0.0
3,0.986696,0.008708,0.004596,0.0
4,0.986696,0.008708,0.004596,0.0


In [ ]:
# 연속형 변수의 결측치를 평균값으로 처리할 수 있도록 SimpleImputer를 사용
from sklearn.impute import SimpleImputer  

# SimpleImputer를 사용할 변수 추출
na_col = ['adult', 'kids', 'teen', 'elder', 'adult_prop', 'kids_prop', 'teen_prop',
          'elder_prop']
# 평균값으로 결측치 처리를 위해 strategy옵션을 mean으로 설정 
imputer = SimpleImputer(strategy="mean")
# 각 변수에 대해 평균값을 구하고 결측치 대치를 위해 fit_transform을 사용
df[na_col] = pd.DataFrame(imputer.fit_transform(df[na_col]), columns = na_col)

#### 3.3.1.5 좌표를 활용한 변수

In [ ]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) # 제주시의 위도 경도
coords_seoquiposi = (33.259429, 126.558217) # 서귀포시의 위도 경도

# 제주시와의 거리
df['dis_jejusi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusi).km for i in range(len(df))]

# 서귀포시와의 거리
df['dis_seoquiposi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_seoquiposi).km for i in range(len(df))]


In [ ]:
df[['dis_jejusi', 'dis_seoquiposi']].head()

* 탑승하는 승객의 수가 많은 버스정류장과의 거리

In [ ]:
# 승차 인원 상위 10개의 버스정류장 좌표
coords_jejusicheong1 = (33.49892, 126.53035) # 제주시청(광양 방면)의 위도 경도
coords_jejuairport = (33.50661, 126.49345) # 제주국제공항(구제주 방면)의 위도 경도
coords_hallahosp = (33.48963, 126.486) # 한라병원의 위도 경도
coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = (33.48902, 126.5392) # 제주중앙여자고등학교의 위도 경도
coords_jejumarket = (33.51315, 126.52706) # 동문시장의 위도 경도
coords_jejusclass = (33.47626, 126.48141) # 제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = (33.51073, 126.5239) # 중앙로(국민은행)의 위도 경도
coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
coords_law = (33.49363, 126.53476) # 제주지방법원(광양 방면)의 위도 경도

# 승차 인원 상위 10개의 버스정류장 좌표와 각 정류장의 지구 표면에서 두 점 사이의 거리를 구함
df['dis_jejusicheong1'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(df))]
df['dis_jejuairport'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(df))]
df['dis_hallahosp'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(df))]
df['dis_rotary'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_rotary).km for i in range(len(df))]
df['dis_jejucenterhigh'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(df))]
df['dis_jejumarket'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(df))]
df['dis_jejusclass'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(df))]
df['dis_centerroad'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_centerroad).km for i in range(len(df))]
df['dis_fiveway'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_fiveway).km for i in range(len(df))]
df['dis_law'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_law).km for i in range(len(df))]

In [ ]:
df[['dis_jejusicheong1', 'dis_jejuairport', 'dis_hallahosp', 'dis_rotary',
    'dis_jejucenterhigh', 'dis_jejumarket', 'dis_jejusclass', 'dis_centerroad',
    'dis_fiveway', 'dis_law']].head()

### 3.3.2. 외부 데이터를 통한 변수 생성

#### 3.3.2.1. 날씨를 활용한 변수

* 버스정류장과 각 측정소의 거리

In [ ]:
import geopy.distance
# 측정소별 위도, 경도
jeju = (33.51411, 126.52969) # 제주 측정소 근처
gosan = (33.29382, 126.16283) # 고산 측정소 근처
seongsan = (33.38677, 126.8802) # 성산 측정소 근처
po = (33.24616, 126.5653) # 서귀포 측정소 근처

# 버스정류장과 제주 측정소와의 거리
t1 = [geopy.distance.geodesic((i,j), jeju).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_jeju'] = t1

# 버스정류장과 고산 측정소와의 거리
t2 = [geopy.distance.geodesic((i,j), gosan).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_gosan']=t2

# 버스정류장과 성산 측정소와의 거리
t3 = [geopy.distance.geodesic((i,j), seongsan).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_seongsan']=t3

# 버스정류장과 서귀포 측정소와의 거리
t4 = [geopy.distance.geodesic((i,j), po).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_po']=t4

In [ ]:
df[['dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po']].head()

,dis_jeju,dis_gosan,dis_seongsan,dis_po
0,4.286659,37.692766,37.713630,27.841879
1,4.966848,37.008492,38.465822,27.996179
2,6.330746,35.642126,39.259820,27.496551
3,3.575272,38.643401,38.386802,29.579404
4,30.652098,23.648361,45.899140,14.269792


* 버스정류장과 가장 가까운 관측소

In [ ]:
total = pd.DataFrame(list(zip(t1,t2,t3,t4)),
                     columns=['jeju', 'gosan', 'seongsan', 'po'])
df['dist_name'] = total.apply(lambda x: x.idxmin(), axis=1)

In [ ]:
df[['dist_name']].head()

,dist_name
0,jeju
1,jeju
2,jeju
3,jeju
4,po


* 지점별 기상정보 변수

In [ ]:
# 변수명 변경 
# ['지점', '일시', '기온(°C)', '강수량(mm)']
rain.columns = ['dist_name', 'date', 'temperature', 'rainfall']

# fillna 함수를 활용해 결측치를 0으로 치환
rain = rain.fillna(0)

# rain 데이터의 date 변수를 datetime 형태로 변환
rain['date'] = pd.to_datetime(rain['date'])

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, rain, how='left', on=['dist_name', 'date'])

In [ ]:
print(df)

In [ ]:
df[['temperature', 'rainfall']].head()

,temperature,rainfall
0,24.650000,0.0
1,24.650000,0.0
2,24.650000,0.0
3,24.650000,0.0
4,23.783333,0.0


* rainy_day

In [ ]:
def is_rainy(x):
    if x == 0:
        return 0 # 비 안오는 날
    else:
        return 1 # 비 오는 날

df['rainy_day'] = df['rainfall'].apply(is_rainy)

In [ ]:
df[['rainy_day']].head()

,rainy_day
0,0
1,0
2,0
3,0
4,0


#### 3.3.2.2. jeju_financial_life_data를 활용한 변수

In [ ]:
# df 주소 데이터(df_location.csv) 로드
df_loc = pd.read_csv("df_location.csv", encoding='cp949')[['location','dong', 'si']]
# jeju_life 주소 데이터(life_location.csv) 로드
life_loc = pd.read_csv("life_location.csv", encoding='cp949'
                      )[['location','dong', 'si']]

# df의 위도, 경도 변수를 문자열로 취급하여 location 변수 생성
df['location'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)
# jeju_life의 위도, 경도 변수를 문자열로 취급하여 location 변수 생성
jeju_life['location'] = jeju_life['y_axis'].astype(str).str[:10] + ',' + \
                        jeju_life['x_axis'].astype(str).str[:10]

# location 변수를 기준으로 df와 df_loc 병합
df = pd.merge(df, df_loc, how='left', on='location')
# location 변수를 기준으로 jeju_life와 life_loc 병합
jeju_life = pd.merge(jeju_life, life_loc, how='left', on='location')

In [ ]:
df[['location', 'dong', 'si']].head()

,location,dong,si
0,"33.4899,126.49373",연동,제주시
1,"33.48944,126.48508",연동,제주시
2,"33.48181,126.47352",노형동,제주시
3,"33.50577,126.49252",용담이동,제주시
4,"33.255790000000005,126.4126",색달동,서귀포시


* 각 동(읍, 면별) 직업, 소득, 소비, 부동산 관련 변수의 평균, 합계, 비율

In [ ]:
col = ['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self', 
       'avg_income', 'med_income', 'avg_spend', 'vehicle_own_rat']

# groupby 함수를 사용해 dong을 기준으로 위에서 선택한 변수들의 평균을 구함
data1 = jeju_life.groupby(['dong'])[col].mean().reset_index()
data1.columns=['dong','mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 
               'mean_job_profession', 'mean_job_self', 'mean_vehicle_own_rat',
               'mean_avg_income', 'mean_med_income', 'mean_avg_spend']

# groupby 함수를 사용해 dong을 기준으로 위에서 선택한 변수들의 합계를 구함 
data2 = jeju_life.groupby(['dong'])[col].sum().reset_index()
data2.columns=['dong','sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 
               'sum_job_profession', 'sum_job_self', 'sum_vehicle_own_rat', 
               'sum_avg_income', 'sum_med_income', 'sum_avg_spend']

# groupby 함수를 사용해 dong을 기준으로 위에서 선택한 변수들의 비율을 구함
data3 = (jeju_life.groupby(['dong'])[col].sum() \
             / jeju_life.groupby(['dong'])[col].sum().sum()).reset_index()
data3.columns = ['dong','rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 
                 'rate_job_profession', 'rate_job_self', 'rate_vehicle_own_rat', 
                 'rate_avg_income', 'rate_med_income', 'rate_avg_spend']

# dong을 기준으로 기존의 데이터프레임에 새로운 변수를 병합
merge_data1 = pd.merge(data1, data2, how='left', on='dong')
merge_data2 = pd.merge(merge_data1, data3, how='left', on='dong')
df = pd.merge(df, merge_data2, how='left', on='dong')

In [ ]:
df[['mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 
    'mean_job_profession', 'mean_job_self', 'mean_vehicle_own_rat', 
    'mean_avg_income', 'mean_med_income', 'mean_avg_spend']].head()

,mean_job_majorc,mean_job_smallc,mean_job_public,mean_job_profession,mean_job_self,mean_vehicle_own_rat,mean_avg_income,mean_med_income,mean_avg_spend
0,0.030512,0.161997,0.039919,0.014888,0.138164,3.511182e+07,3.095374e+07,4.683326e+06,0.031514
1,0.030512,0.161997,0.039919,0.014888,0.138164,3.511182e+07,3.095374e+07,4.683326e+06,0.031514
2,0.032494,0.154253,0.042997,0.020755,0.139257,3.678985e+07,3.265628e+07,5.128032e+06,0.035210
3,0.025593,0.167158,0.035757,0.014249,0.135443,3.076244e+07,2.891900e+07,4.000557e+06,0.029785
4,0.035615,0.178596,0.019174,0.004394,0.305618,3.498224e+07,3.017862e+07,4.115050e+06,0.044791


In [ ]:
df[['sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 
    'sum_job_profession', 'sum_job_self', 'sum_vehicle_own_rat', 
    'sum_avg_income', 'sum_med_income', 'sum_avg_spend']].head()

,sum_job_majorc,sum_job_smallc,sum_job_public,sum_job_profession,sum_job_self,sum_vehicle_own_rat,sum_avg_income,sum_med_income,sum_avg_spend
0,18.948217,100.600402,24.789940,9.245359,85.799611,2.180444e+10,1.922227e+10,2.908346e+09,19.570112
1,18.948217,100.600402,24.789940,9.245359,85.799611,2.180444e+10,1.922227e+10,2.908346e+09,19.570112
2,19.918703,94.556852,26.357053,12.723008,85.364807,2.255218e+10,2.001830e+10,3.143484e+09,21.583853
3,6.244585,40.786531,8.724751,3.476796,33.048145,7.506035e+09,7.056235e+09,9.761358e+08,7.267512
4,1.032846,5.179291,0.556057,0.127439,8.862915,1.014485e+09,8.751800e+08,1.193365e+08,1.298941


In [ ]:
df[['rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 
    'rate_job_profession', 'rate_job_self', 'rate_vehicle_own_rat', 
    'rate_avg_income', 'rate_med_income', 'rate_avg_spend']].head()

In [ ]:
# 새로 생성한 27개의 변수 정보
df[merge_data2.columns[1:]].info() # 새로 생성한 27개 변수 각각 201개의 NA값이 존재

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643593 entries, 0 to 643592
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   mean_job_majorc       641868 non-null  float64
 1   mean_job_smallc       641868 non-null  float64
 2   mean_job_public       641868 non-null  float64
 3   mean_job_profession   641868 non-null  float64
 4   mean_job_self         641868 non-null  float64
 5   mean_vehicle_own_rat  641868 non-null  float64
 6   mean_avg_income       641868 non-null  float64
 7   mean_med_income       641868 non-null  float64
 8   mean_avg_spend        641868 non-null  float64
 9   sum_job_majorc        641868 non-null  float64
 10  sum_job_smallc        641868 non-null  float64
 11  sum_job_public        641868 non-null  float64
 12  sum_job_profession    641868 non-null  float64
 13  sum_job_self          641868 non-null  float64
 14  sum_vehicle_own_rat   641868 non-null  float64
 15  

In [ ]:
# 결측치를 평균값으로 처리할 수 있도록 SimpleImputer를 사용
imputer = SimpleImputer(strategy="mean")

# 결측치가 존재하는 변수 목록
na_col = list(merge_data2.columns[1:])

# 각 변수에 대해 평균값을 구하고 결측치 대치를 위해 fit_transform을 사용
df[na_col] = pd.DataFrame(imputer.fit_transform(df[na_col]), columns = na_col)

In [ ]:
# NA값 확인
df[na_col].isnull().sum()

mean_job_majorc         0
mean_job_smallc         0
mean_job_public         0
mean_job_profession     0
mean_job_self           0
mean_vehicle_own_rat    0
mean_avg_income         0
mean_med_income         0
mean_avg_spend          0
sum_job_majorc          0
sum_job_smallc          0
sum_job_public          0
sum_job_profession      0
sum_job_self            0
sum_vehicle_own_rat     0
sum_avg_income          0
sum_med_income          0
sum_avg_spend           0
rate_job_majorc         0
rate_job_smallc         0
rate_job_public         0
rate_job_profession     0
rate_job_self           0
rate_vehicle_own_rat    0
rate_avg_income         0
rate_med_income         0
rate_avg_spend          0
dtype: int64

### 3.3.3. 추가 변수 생성

<font color='blue'>3.3.3. '추가 변수 생성'에서 생성하는 변수는 책에는 나와있지 않으므로 주피터 파일을 참고해주세요.

* 승하차 시간대 통합 변수(t~t+3)

In [ ]:
# 오전시간대에 3시간동안 승차한 인원수의 합계
df['69a'] = df['6~7_ride'] + df['7~8_ride'] + df['8~9_ride']
df['912a']= df['9~10_ride'] + df['10~11_ride'] + df['11~12_ride']

# 오전시간대에 3시간동안 하차한 인원수의 합계
df['69b'] = df['6~7_takeoff'] + df['7~8_takeoff'] + df['8~9_takeoff']
df['912b'] = df['9~10_takeoff'] + df['10~11_takeoff'] + df['11~12_takeoff']

In [ ]:
df[['69a', '912a', '69b', '912b']].head()

,69a,912a,69b,912b
0,3.0,13.0,0.0,0.0
1,9.0,13.0,0.0,0.0
2,2.0,2.0,0.0,0.0
3,23.0,56.0,0.0,0.0
4,0.0,0.0,0.0,1.0


* 제주도 기상정보 변수

In [ ]:
# 변수명 변경
# ['날짜', '현재일기', '현재기온', '체감온도', '일강수']
weather.columns = ['date','description','temp','feels_like','rain']
weather['rain'] = weather['rain'].replace(' ', '0')
weather['rain'] = weather['rain'].astype(float)
weather['date'] = pd.to_datetime(weather['date'])
df = pd.merge(df, weather, how='left', on='date')

In [ ]:
df[['description', 'temp', 'feels_like', 'rain']].head()

,description,temp,feels_like,rain
0,흐림,26.2,75.0,0.0
1,흐림,26.2,75.0,0.0
2,흐림,26.2,75.0,0.0
3,흐림,26.2,75.0,0.0
4,흐림,26.2,75.0,0.0


* id별 퇴근 시간 총/평균 승객수

In [ ]:
def id_statistic(ID, col1, col2) :
    
    train_data = df.query('cue==0').reset_index()    

    # mean, sum
    rs_mean = train_data.groupby([ID])['18~20_ride'].agg([(col1, 'mean')]
                                                        ).reset_index()
    rs_sum = train_data.groupby([ID])['18~20_ride'].agg([(col2, 'sum')]
                                                       ).reset_index()
    rs_mean_sum = pd.merge(rs_mean, rs_sum, on=ID)

    # merge
    merge_data = pd.merge(df, rs_mean_sum, how='left', on=ID)

    # 연속형 변수의 결측치를 평균값으로 대체
    merge_data[col1] = merge_data[col1].fillna(rs_mean[col1].mean())
    merge_data[col2] = merge_data[col2].fillna(rs_sum[col2].mean())
    
    return merge_data


df = id_statistic('bus_route_id', '1820_r_mean', '1820_r_sum')
df = id_statistic('station_code', '1820_s_mean', '1820_s_sum')
df = id_statistic('weekday', '1820_w_mean', '1820_w_sum')
df = id_statistic('route_station', '1820_rs_mean', '1820_rs_sum')

In [ ]:
df[['1820_r_mean', '1820_r_sum', '1820_s_mean', '1820_s_sum',
   '1820_w_mean', '1820_w_sum', '1820_rs_mean', '1820_rs_sum']].head()

,1820_r_mean,1820_r_sum,1820_s_mean,1820_s_sum,1820_w_mean,1820_w_sum,1820_rs_mean,1820_rs_sum
0,3.104381,2409.0,1.466667,44.0,1.034282,54306.0,1.466667,44.0
1,3.104381,2409.0,4.178218,6330.0,1.034282,54306.0,5.366667,161.0
2,3.104381,2409.0,2.169559,1625.0,1.034282,54306.0,1.600000,48.0
3,3.104381,2409.0,52.032258,1613.0,1.034282,54306.0,53.766667,1613.0
4,3.104381,2409.0,0.732794,181.0,1.034282,54306.0,0.153846,4.0


* 라벨 인코딩

In [ ]:
# sklearn라이브러리에서 라벨 인코딩 객체 불러와서 저장하기
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

# 라벨 인코딩 할 변수 선택
df_encode = df[['description', 'bus_route_id','station_code']]
# fit_transform을 통해 라벨 인코딩 
df_encoded = df_encode.apply(labelencoder.fit_transform)

df['description'] = df_encoded['description']
df['bus_route_id2'] = df_encoded['bus_route_id']
df['station_code2'] = df_encoded['station_code']

In [ ]:
df[['description', 'bus_route_id2', 'station_code2']].head()

,description,bus_route_id2,station_code2
0,9,625,322
1,9,625,335
2,9,625,408
3,9,625,1448
4,9,625,1510


### 3.3.4. 라벨 인코딩과 원핫 인코딩 변수

#### 3.3.4.1. 라벨 인코딩 변수

* in_out

In [ ]:
df['in_out'] = df['in_out'].map({'시내':0, '시외':1})

In [ ]:
df[['in_out']].head()

,in_out
0,1
1,1
2,1
3,0
4,0


* 주말 및 주중

In [ ]:
def is_weekend(x):
    if x == 5: # 토요일
        return 1
    elif x == 6: # 일요일
        return 1
    else:
        return 0
# 'weekday’변수에 is_weekend함수를 적용하여 'weekend' 변수 생성      
df['weekend'] = df['weekday'].apply(is_weekend)

In [ ]:
df[['weekend']].head()

,weekend
0,1
1,1
2,1
3,1
4,1


* 시, 동

In [ ]:
labelencoder = LabelEncoder()

# 라벨 인코딩 할 변수 선택
df_encode = df[['si', 'dong']]
# fit_transform을 통해 라벨 인코딩 
df_encoded = df_encode.apply(labelencoder.fit_transform)

df['si']=df_encoded['si']
df['dong']=df_encoded['dong']

In [ ]:
df[['si', 'dong']].head()

,si,dong
0,1,36
1,1,36
2,1,5
3,1,47
4,0,26


#### 3.3.4.2. 원핫 인코딩 변수

In [ ]:
df = pd.get_dummies(df, columns=['weekday', 'dist_name'])

In [ ]:
df[['weekday_0', 'weekday_1', 'weekday_2', 'weekday_3',
    'weekday_4', 'weekday_5', 'weekday_6']].head()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [ ]:
df[['dist_name_gosan','dist_name_jeju','dist_name_po','dist_name_seongsan']].head()

,dist_name_gosan,dist_name_jeju,dist_name_po,dist_name_seongsan
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,0,1,0


In [ ]:
df.shape

(643593, 131)

## 모델구축과 검증


In [ ]:
# 문자 형태의 변수 추출
object_column = list(df.columns[df.dtypes == 'object'])
# 문자형 변수 제거
df = df.drop(object_column, axis='columns')

# datetime 형태의 변수 추출
datetime_column = list(df.columns[df.dtypes == 'datetime64[ns]'])
# datetime형 변수 제거
#df = df.drop(datetime_column, axis='columns')

In [ ]:
print(df)

            id       date  in_out  station_code  latitude  longitude  \
0            0 2019-09-01       1           344  33.48990  126.49373   
1            1 2019-09-01       1           357  33.48944  126.48508   
2            2 2019-09-01       1           432  33.48181  126.47352   
3            3 2019-09-01       0          1579  33.50577  126.49252   
4            4 2019-09-01       0          1646  33.25579  126.41260   
...        ...        ...     ...           ...       ...        ...   
643588  643588 2019-10-16       0           786  33.30073  126.18044   
643589  643589 2019-10-16       0          1080  33.46262  126.33447   
643590  643590 2019-10-16       0          1129  33.41437  126.26336   
643591  643591 2019-10-16       0          1564  33.49946  126.51479   
643592  643592 2019-10-16       0       6115100  33.24873  126.50799   

        6~7_ride  7~8_ride  8~9_ride  9~10_ride  ...  weekday_1  weekday_2  \
0            0.0       1.0       2.0        5.0  ...     

In [ ]:
print(object_column + datetime_column)
print(object_column)
print(datetime_column)

['bus_route_id', 'station_name', 'route_station', 'location', 'date']
['bus_route_id', 'station_name', 'route_station', 'location']
['date']


In [ ]:
df.shape

(643593, 19)

In [ ]:
# 학습 데이터
X_train = df.query("cue==0").drop(['cue', '18~20_ride'], axis='columns')
# 테스트 데이터
X_test = df.query("cue==1").drop(['cue', '18~20_ride'], axis='columns')
# 타깃 변수
y_train = df.query("cue==0")['18~20_ride']

print(X_train)

### 유가 데이터 추가 후 모델 구축  
많은 요소 중 유가 정보가 퇴근시간 버스 승차인원 예측에 영향을 줄 수 있다고 생각한다. 날짜별 유가 정보를 제공해주는 사이트인 Opinet을 이용하여 2019년 9월 1일 ~ 2019년 10월 16일 제주도 주유소 평균 휘발유, 경유 판매가격(￦)을 이용한다.  
https://www.opinet.co.kr/user/dopospdrg/dopOsPdrgAreaSelect.do

In [10]:
oil = pd.read_csv('oil.csv')
oil

,date,diesel,gasoline
0,2019-09-01,1424.71,1593.94
1,2019-09-02,1426.13,1595.71
2,2019-09-03,1425.81,1594.92
3,2019-09-04,1424.77,1593.05
4,2019-09-05,1424.69,1592.56
5,2019-09-06,1457.01,1612.38
6,2019-09-07,1458.66,1613.46
7,2019-09-08,1459.28,1614.42
8,2019-09-09,1459.23,1614.07
9,2019-09-10,1460.05,1614.80


In [ ]:
oil2 = pd.merge(df,oil, how='left', on= 'date')
oil2.to_csv('oil2.csv', index = False, encoding = 'utf-8')

In [ ]:
# 학습 데이터
X_train3 = oil2.query("cue==0").drop(['cue','date', '18~20_ride'], axis='columns')
# 테스트 데이터
X_test3 = oil2.query("cue==1").drop(['cue','date', '18~20_ride'], axis='columns')
# 타깃 변수
y_train3 = oil2.query("cue==0")['18~20_ride']


In [14]:
oil2 = pd.read_csv('oil2.csv')
print(oil2)

            id        date  in_out  station_code  latitude  longitude  \
0            0  2019-09-01       1           344  33.48990  126.49373   
1            1  2019-09-01       1           357  33.48944  126.48508   
2            2  2019-09-01       1           432  33.48181  126.47352   
3            3  2019-09-01       0          1579  33.50577  126.49252   
4            4  2019-09-01       0          1646  33.25579  126.41260   
...        ...         ...     ...           ...       ...        ...   
643588  643588  2019-10-16       0           786  33.30073  126.18044   
643589  643589  2019-10-16       0          1080  33.46262  126.33447   
643590  643590  2019-10-16       0          1129  33.41437  126.26336   
643591  643591  2019-10-16       0          1564  33.49946  126.51479   
643592  643592  2019-10-16       0       6115100  33.24873  126.50799   

        6~7_ride  7~8_ride  8~9_ride  9~10_ride  ...  weekday_3  weekday_4  \
0            0.0       1.0       2.0        5

In [ ]:
X_train3.to_csv('X_train3.csv', index = False, encoding = 'utf-8')
X_test3.to_csv('X_test3.csv', index = False, encoding = 'utf-8')
y_train3.to_csv('y_train3.csv', index = False, encoding = 'utf-8')

print('save ..!')

In [ ]:
print(X_test3)

            id  in_out  station_code  latitude  longitude  6~7_ride  7~8_ride  \
0       415423       1           344  33.48990  126.49373       4.0       4.0   
1       415424       1           357  33.48944  126.48508       1.0       6.0   
2       415425       1           432  33.48181  126.47352       2.0       4.0   
3       415426       0          1579  33.50577  126.49252       1.0      11.0   
4       415427       0          1636  33.24872  126.41032       0.0       0.0   
...        ...     ...           ...       ...        ...       ...       ...   
228165  643588       0           786  33.30073  126.18044       0.0       0.0   
228166  643589       0          1080  33.46262  126.33447       0.0       0.0   
228167  643590       0          1129  33.41437  126.26336       3.0       0.0   
228168  643591       0          1564  33.49946  126.51479       3.0       0.0   
228169  643592       0       6115100  33.24873  126.50799       0.0       0.0   

        8~9_ride  9~10_ride

In [ ]:
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)

(415423, 17)

(228170, 17)

(415423,)

### 3.4.1. 머신러닝 모델

* 랜덤포레스트

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=1217)
# 모델 학습
rf_model.fit(X_train, y_train)
# 예측값 생성
y_test = rf_model.predict(X_test)

* Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

adb_model = AdaBoostRegressor(random_state=1217)

# 모델 학슴
adb_model.fit(X_train, y_train)

# 예측값 생성
y_test = adb_model.predict(X_test)


* XGBoost

In [ ]:
%%time

from xgboost import XGBRegressor

xgb_model = XGBRegressor(random_state=1217)
# 모델 학습
xgb_model.fit(X_train, y_train)
# 예측값 생성
y_test = xgb_model.predict(X_test)

#print(y_test)

[06:05:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 32.2 ms, sys: 991 µs, total: 33.2 ms
Wall time: 34.1 ms


*  Gradient Tree Boosting

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingRegressor

gdb_model = GradientBoostingRegressor(random_state=1217)
# 모델 학습
gdb_model.fit(X_train3, y_train3)
# 예측값 생성
y_test3 = gdb_model.predict(X_test3)

CPU times: user 11min 13s, sys: 983 ms, total: 11min 14s
Wall time: 11min 13s


* LightGBM

In [ ]:
%%time

import lightgbm as lgb

lgbm = lgb.LGBMRegressor(random_state=1217)
# 모델 학습
lgbm.fit(X_train3, y_train3)
# 예측값 생성
y_test3 = lgbm.predict(X_test3)

CPU times: user 35.3 s, sys: 624 ms, total: 35.9 s
Wall time: 20.7 s


### 3.4.2. 모델 검증

#### 3.4.2.1. 교차검증

In [ ]:
%%time

from sklearn.model_selection import cross_val_score

rf_model = RandomForestRegressor(random_state=1217)

# cv=5인 교차 검증
scores = cross_val_score(rf_model, X_train, y_train, cv=5, n_jobs=-1,
                         scoring = 'neg_mean_squared_error')

# 성능 확인
print('cross_val_score \n{}'.format(np.sqrt(-scores)))
print('cross_val_score.mean \n{:.3f}'.format(np.sqrt(-scores.mean())))

### 3.4.3. 변수 선택

#### 3.4.3.1. 최종 데이터 구축

In [ ]:
input_var1=['in_out', 'latitude', 'longitude','6~7_ride', '7~8_ride', '8~9_ride',
            '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
            '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
            'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
            'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_jejusicheong1',
            'dis_jejuairport', 'dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
            'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway',
            'dis_law', 'weekend', 'holiday', 'ride_sum', 'takeoff_sum',
            '1820_r_mean', '1820_s_mean', '1820_w_mean', '1820_rs_mean',
            'station_code2', 'bus_route_id2', 'rain', 'description', 'feels_like',
            '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum',
            '10~11_ride_sum', 'in_out_mean', 'weekdaymean', 'adult', 'kids', 'teen',
            'elder', 'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop',
            'mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 
            'mean_job_profession', 'mean_job_self', 'mean_vehicle_own_rat',
            'mean_avg_income', 'mean_med_income', 'mean_avg_spend',
            'rate_job_majorc', 'rate_job_smallc', 'rate_job_public',
            'rate_job_profession', 'rate_job_self', 'rate_vehicle_own_rat', 
            'rate_avg_income', 'rate_med_income', 'rate_avg_spend',
            'sum_job_majorc', 'sum_job_smallc', 'sum_job_public',
            'sum_job_profession', 'sum_job_self', 'sum_vehicle_own_rat',
            'sum_avg_income', 'sum_med_income', 'sum_avg_spend', '68a', '810a',
            '1012a', '68b', '810b', '1012b', '69a', '912a', '69b', '912b',
            'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po', 'dist_name_gosan',
            'dist_name_jeju', 'dist_name_po', 'dist_name_seongsan', 'temperature',
            'rainfall', 'school', 'transfer', 'si', 'dong', 'rainy_day','휘발유가격','경유가격']

In [ ]:
input_var2=['in_out', 'latitude', 'longitude','6~7_ride', '7~8_ride', '8~9_ride',
            '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
            '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
            'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 
            'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_jejusicheong1',
            'dis_jejuairport','dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
            'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway',
            'dis_law', 'weekend', 'holiday', 'ride_sum', 'takeoff_sum',
            '1820_rs_mean', '1820_r_mean', '1820_s_mean', 'station_code2',
            'bus_route_id2', 'rain', 'description', 'feels_like', '1820_w_mean',
            'in_out_mean', 'weekdaymean', 'adult', 'kids', 'teen', 'elder',
            'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop', 'mean_job_majorc',
            'mean_job_smallc', 'mean_job_public', 'mean_job_profession',
            'mean_job_self', 'mean_vehicle_own_rat', '68a', '810a', '1012a', '68b',
            '810b', '1012b', 'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po',
            'dist_name_gosan','dist_name_jeju', 'dist_name_po', 'dist_name_seongsan',
            'temperature', 'rainfall','휘발유가격','경유가격']

In [ ]:
input_var3=['in_out', 'latitude', 'longitude','6~7_ride', '7~8_ride', '8~9_ride',
            '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
            '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
            'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
            'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_jejuairport', 'ride_sum',
            'takeoff_sum','1820_r_mean','1820_r_sum', '1820_s_mean', '1820_s_sum', 
            '1820_w_mean', '1820_w_sum', '1820_rs_mean', '1820_rs_sum',
            'bus_route_id2', '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum',
            '9~10_ride_sum', '10~11_ride_sum', 'sum_avg_spend', '68a', '810a', 'si',
            'dong','휘발유가격','경유가격']

### 3.4.4. 하이퍼파라미터 튜닝

#### 3.4.4.1. 그리드 서치

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state=1217)

# 탐색할 하이퍼파라미터 값 지정
params = {'n_estimators' : [200, 300, 500],
          'max_features': [5, 6, 8],
          'min_samples_leaf' : [1, 3, 5]}

# GridSearchCV를 이용해 최적의 하이퍼파라미터 탐색
grid_cv = GridSearchCV(rf, param_grid = params, cv = 3, n_jobs=-1)
grid_cv.fit(X_train[input_var1], y_train)

# 최적의 하이퍼파라미터 값과 성능 출력
print('최적 하이퍼파라미터: ', grid_cv.best_params_)
print('최적 하이퍼파라미터의 성능(RMSE): {:.4f}'.format(
    np.sqrt(grid_cv.best_score_)))

#### 3.4.4.2. 랜덤 서치

In [ ]:
%%time

import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor(random_state=1217)

# 탐색할 하이퍼파라미터 구간 지정
params = {'n_estimators': stats.randint(200, 250),
          'max_features': stats.randint(6, 8),
          'min_samples_leaf': stats.randint(1, 3)}

# RandomizedSearchCV를 이용해 최적의 하이퍼파라미터 탐색
rand_cv = RandomizedSearchCV(rf, param_distributions = params, n_iter=10, 
                             cv = 3, random_state = 1217, n_jobs=-1)
rand_cv.fit(X_train[input_var1], y_train)

# 최적의 하이퍼파라미터 값과 성능 출력
print('최적 하이퍼파라미터: ', rand_cv.best_params_)
print('최적 하이퍼파라미터의 성능(RMSE): {:.4f}'.format(np.sqrt(rand_cv.best_score_)))

### 3.4.5. 최종 모델 구축

#### 3.4.5.2. 최종 모델 구축

In [ ]:
# submission_sample 데이터 로드
sub = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/week4/ch03/submission_sample.csv')

In [ ]:
sub_dir = "/content/gdrive/My Drive/Colab Notebooks/week4/ch03/submission2/12/" 

# submission 폴더가 없다면 submission 폴더를 생성
if not os.path.isdir(sub_dir):
    os.mkdir(sub_dir)

# 작업 경로를 submission 폴더로 변경
os.chdir(sub_dir)

* 첫 번째 데이터로 만든 모델

In [ ]:
%%time

# 첫 번째 모델 정의
model1 = lgb.LGBMRegressor(random_state=1217, num_iterations = 1000, learning_rate = 0.05,
                           boosting = 'dart', Metric = 'regression_l2', n_jobs=-1)
# 모델 학습
model1.fit(X_train3[input_var1], y_train3)
# 예측값 생성
pred1 = model1.predict(X_train3[input_var1])
# 성능 확인
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train3, pred1))

CPU times: user 18min 46s, sys: 1.76 s, total: 18min 48s
Wall time: 9min 36s


In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred1 = model1.predict(X_test3[input_var1])
sub['18~20_ride'] = y_pred1
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model1_lgbm.csv', index = False)

In [ ]:
%%time

# 두 번째 모델 정의
model2 = RandomForestRegressor(random_state=1217, max_features=8, 
                               min_samples_leaf=1, n_estimators=500, n_jobs=-1)
# 모델 학습
model2.fit(X_train3[input_var1], y_train3)
# 예측값 생성
pred2 = model2.predict(X_train3[input_var1])
# 성능 확인
np.sqrt(mean_squared_error(y_train3, pred2))

CPU times: user 25min 47s, sys: 19.8 s, total: 26min 7s
Wall time: 14min 30s


In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred2 = model2.predict(X_test3[input_var1])
sub['18~20_ride'] = y_pred2
# 'submission' 폴더에 CSV 파일 저장
sub.to_csv('model2_rf.csv', index = False)

In [ ]:
%%time

# 세 번째 모델 정의
model3 = RandomForestRegressor(random_state=1217, max_features=3, 
                               min_samples_leaf=2, n_estimators=500, n_jobs=-1)
# 모델 학습
model3.fit(X_train3[input_var1], y_train3)
# 예측값 생성
pred3 = model3.predict(X_train3[input_var1])
# 성능 확인
np.sqrt(mean_squared_error(y_train3, pred3))

In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred3 = model3.predict(X_test3[input_var1])
sub['18~20_ride'] = y_pred3
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model3_rf.csv', index = False)

* 두 번째 데이터로 만든 모델

In [ ]:
%%time

# 네 번째 모델 정의
model4 = RandomForestRegressor(random_state=1217, max_features=4, 
                               min_samples_leaf=2, n_estimators=500, n_jobs=-1)
# 모델 학습
model4.fit(X_train3[input_var2], y_train3)
# 예측값 생성
pred4 = model4.predict(X_train3[input_var2])
# 성능 확인
np.sqrt(mean_squared_error(y_train3, pred4))

CPU times: user 13min 29s, sys: 8.6 s, total: 13min 38s
Wall time: 7min 10s


In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred4 = model4.predict(X_test3[input_var2])
sub['18~20_ride'] = y_pred4
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model4_rf.csv', index = False)

* 세 번째 데이터로 만든 모델

In [ ]:
%%time

# 다섯 번째 모델 정의
model5 = RandomForestRegressor(random_state=1217, max_features=7, 
                               min_samples_leaf=4, n_estimators=300, n_jobs=-1)
# 모델 학습
model5.fit(X_train3[input_var3], y_train3)
# 예측값 생성
pred5 = model5.predict(X_train3[input_var3])
# 성능 확인
np.sqrt(mean_squared_error(y_train3, pred5))

CPU times: user 9min 26s, sys: 2.98 s, total: 9min 29s
Wall time: 5min


In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred5 = model5.predict(X_test3[input_var3])
sub['18~20_ride'] = y_pred5
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model5_rf.csv', index = False)

* AdaBoost

In [ ]:
%%time

# 1번째 데이터로 AdaBoost 모델 정의
model8 = AdaBoostRegressor (base_estimator = None, n_estimators = 50, learning_rate = 1.0 ,
                   loss = 'linear' , random_state = None )

# 모델 학습
model8.fit(X_train[input_var1], y_train)
# 예측값 생성
pred8 = model8.predict(X_train[input_var1])
# 성능 확인
# from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train, pred8))

CPU times: user 6min 35s, sys: 2.82 s, total: 6min 37s
Wall time: 6min 38s


In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred8 = model8.predict(X_test[input_var1])
sub['18~20_ride'] = y_pred8
# 'submission2' 폴더에 csv 파일 저장
sub.to_csv('model8_adb_var1.csv', index = False)

In [ ]:
%%time

# 2번째 데이터로 AdaBoost 모델 정의
model9 = AdaBoostRegressor (base_estimator = None, n_estimators = 50, learning_rate = 1.0 ,
                   loss = 'linear' , random_state = None )

# 모델 학습
model9.fit(X_train[input_var2], y_train)
# 예측값 생성
pred9 = model9.predict(X_train[input_var2])
# 성능 확인
# from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train, pred9))

CPU times: user 4min 45s, sys: 1.19 s, total: 4min 46s
Wall time: 5min 7s


In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred9 = model9.predict(X_test[input_var2])
sub['18~20_ride'] = y_pred9
# 'submission2' 폴더에 csv 파일 저장
sub.to_csv('model9_adb_var2.csv', index = False)

In [ ]:
%%time

# 3번째 데이터로 AdaBoost 모델 정의
model10 = AdaBoostRegressor (base_estimator = None, n_estimators = 50, learning_rate = 1.0 ,
                   loss = 'linear' , random_state = None )

# 모델 학습
model10.fit(X_train[input_var3], y_train)
# 예측값 생성
pred10 = model10.predict(X_train[input_var3])
# 성능 확인
# from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train, pred10))

CPU times: user 2min 20s, sys: 715 ms, total: 2min 21s
Wall time: 2min 23s


In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred10 = model10.predict(X_test[input_var3])
sub['18~20_ride'] = y_pred10
# 'submission2' 폴더에 csv 파일 저장
sub.to_csv('model9_adb_var3.csv', index = False)

* Gradient Tree Boosting으로 만든 모델

* 첫번째

In [ ]:
%time
from sklearn.metrics import mean_squared_error

# Gradient Tree Boosting 모델 정의
model11 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, 
                                    max_depth=1, random_state=0, loss='squared_error')
# 모델 학습
model11.fit(X_train3[input_var1], y_train3)
# 예측값 생성
pred11 = model11.predict(X_train3[input_var1])
# 성능 확인

np.sqrt(mean_squared_error(y_train3, pred11))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs


2.5271587911769564

In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred11 = model11.predict(X_test3[input_var1])
sub['18~20_ride'] = y_pred11
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model11_gdb.csv', index = False)

* 두번째

In [ ]:
%time
from sklearn.metrics import mean_squared_error

# Gradient Tree Boosting 모델 정의
model12 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, 
                                    max_depth=3, random_state=0, loss='squared_error')
# 모델 학습
model12.fit(X_train3[input_var1], y_train3)
# 예측값 생성
pred12 = model12.predict(X_train3[input_var1])
# 성능 확인

np.sqrt(mean_squared_error(y_train3,pred12))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


2.0474176066578993

In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred12 = model12.predict(X_test3[input_var1])
sub['18~20_ride'] = y_pred12
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model12_gdb.csv', index = False)

* 세번째

In [ ]:
%time
from sklearn.metrics import mean_squared_error

# Gradient Tree Boosting 모델 정의
model13 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, 
                                    max_depth=5, random_state=0, loss='squared_error')
# 모델 학습
model13.fit(X_train3[input_var1], y_train3)
# 예측값 생성
pred13 = model13.predict(X_train3[input_var1])
# 성능 확인

np.sqrt(mean_squared_error(y_train3,pred13))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs


1.8008602434074186

In [ ]:
# X_test 데이터에 대한 예측값 생성
y_pred13 = model13.predict(X_test3[input_var1])
sub['18~20_ride'] = y_pred13
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model13_gdb.csv', index = False)